In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%pylab inline
plt.rcParams['figure.figsize']=10,7.5
plt.rcParams['axes.grid']=True
plt.gray()

Populating the interactive namespace from numpy and matplotlib


<Figure size 720x540 with 0 Axes>

In [7]:
from sklearn.preprocessing import imputation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans,DBSCAN
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor

In [9]:
%cd D:\ANALYTIXLAB\PYTHON\Class 11 02-3-19\Class 11and12 files

D:\ANALYTIXLAB\PYTHON\Class 11 02-3-19\Class 11and12 files


In [11]:
car_sales=pd.read_csv("car_sales.csv")
car_sales.head(2)

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
0,Acura,Integra,16.919,16.360,Passenger,21.5,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2/2/2012,58.280150
1,Acura,TL,39.384,19.875,Passenger,28.4,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,6/3/2011,91.370778


In [12]:
car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 16 columns):
Manufacturer           157 non-null object
Model                  157 non-null object
Sales_in_thousands     157 non-null float64
__year_resale_value    121 non-null float64
Vehicle_type           157 non-null object
Price_in_thousands     155 non-null float64
Engine_size            156 non-null float64
Horsepower             156 non-null float64
Wheelbase              156 non-null float64
Width                  156 non-null float64
Length                 156 non-null float64
Curb_weight            155 non-null float64
Fuel_capacity          156 non-null float64
Fuel_efficiency        154 non-null float64
Latest_Launch          157 non-null object
Power_perf_factor      155 non-null float64
dtypes: float64(12), object(4)
memory usage: 19.7+ KB


In [13]:
car_sales.columns

Index([u'Manufacturer', u'Model', u'Sales_in_thousands',
       u'__year_resale_value', u'Vehicle_type', u'Price_in_thousands',
       u'Engine_size', u'Horsepower', u'Wheelbase', u'Width', u'Length',
       u'Curb_weight', u'Fuel_capacity', u'Fuel_efficiency', u'Latest_Launch',
       u'Power_perf_factor'],
      dtype='object')

### Method 1:

#### Separate Numerical and Categorical variables:

In [15]:
numeric_var=[key for key in dict(car_sales.dtypes) if dict(car_sales.dtypes)[key] in ['float64','int64','float32','int32']]
cat_var=[key for key in dict(car_sales.dtypes) if dict(car_sales.dtypes)[key] in ['object']]
print(numeric_var)
print(cat_var)

['Curb_weight', 'Engine_size', 'Fuel_efficiency', 'Price_in_thousands', 'Power_perf_factor', '__year_resale_value', 'Fuel_capacity', 'Wheelbase', 'Width', 'Length', 'Horsepower', 'Sales_in_thousands']
['Latest_Launch', 'Model', 'Vehicle_type', 'Manufacturer']


#### DataFrame for Categorical and Numerical :

In [46]:
#DataFrame for Numerical and Categorical variable:
car_sales_num=car_sales[numeric_var]
car_sales_cat=car_sales[cat_var]


In [21]:
car_sales_num.head(2)

,Curb_weight,Engine_size,Fuel_efficiency,Price_in_thousands,Power_perf_factor,__year_resale_value,Fuel_capacity,Wheelbase,Width,Length,Horsepower,Sales_in_thousands
0,2.639,1.8,28.0,21.5,58.280150,16.360,13.2,101.2,67.3,172.4,140.0,16.919
1,3.517,3.2,25.0,28.4,91.370778,19.875,17.2,108.1,70.3,192.9,225.0,39.384


In [22]:
car_sales_cat.head(2)

,Latest_Launch,Model,Vehicle_type,Manufacturer
0,2/2/2012,Integra,Passenger,Acura
1,6/3/2011,TL,Passenger,Acura


#### Creating Summary

In [24]:
#Creating Data Audit Report for Numerical
#Use general function that returns multiple values:
def var_summary(x):
    return pd.Series([x.count(),x.isnull().sum(),x.sum(),x.mean(),x.median(),x.std(),x.var(),x.min(),x.dropna().quantile(.01),
                     x.dropna().quantile(.05),x.dropna().quantile(.10),x.dropna().quantile(.25),x.dropna().quantile(.50),
                     x.dropna().quantile(.75),x.dropna().quantile(.90),x.dropna().quantile(.95),x.dropna().quantile(.99),
                     x.max()],index=['N','NMISS','SUM','MEAN','MEDIAN','STD','VAR','MIN','P1','P5','P10','P25','P50','P75',
                                    'P90','P95','P99','MAX'])
num_summary=car_sales_num.apply(lambda x:var_summary(x)).T
num_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Curb_weight,155.0,2.0,523.594000,3.378026,3.342000,0.630502,0.397532,1.895000,2.24540,2.423500,2.606200,2.971000,3.342000,3.799500,4.118600,4.389100,5.396680,5.572000
Engine_size,156.0,1.0,477.500000,3.060897,3.000000,1.044653,1.091300,1.000000,1.55500,1.800000,1.950000,2.300000,3.000000,3.575000,4.600000,4.775000,5.700000,8.000000
Fuel_efficiency,154.0,3.0,3672.000000,23.844156,24.000000,4.282706,18.341567,15.000000,15.00000,16.650000,18.300000,21.000000,24.000000,26.000000,28.700000,31.000000,33.000000,45.000000
Price_in_thousands,155.0,2.0,4245.567000,27.390755,22.799000,14.351653,205.969949,9.235000,10.23144,12.469000,13.970800,18.017500,22.799000,31.947500,44.017000,55.835000,78.479800,85.500000
Power_perf_factor,155.0,2.0,11941.756636,77.043591,72.030917,25.142664,632.153558,23.276272,38.46192,46.203997,48.447741,60.407707,72.030917,89.414878,108.499252,125.091513,141.119461,188.144323
__year_resale_value,121.0,36.0,2186.830000,18.072975,14.180000,11.453384,131.180007,5.160000,6.17300,7.850000,9.125000,11.260000,14.180000,19.875000,32.075000,41.250000,60.220000,67.550000
Fuel_capacity,156.0,1.0,2800.500000,17.951923,17.200000,3.887921,15.115932,10.300000,11.90000,12.500000,13.200000,15.800000,17.200000,19.575000,23.200000,25.400000,30.900000,32.000000
Wheelbase,156.0,1.0,16768.000000,107.487179,107.000000,7.641303,58.389512,92.600000,92.87500,95.875000,98.650000,103.000000,107.000000,112.200000,115.500000,119.250000,134.375000,138.700000
Width,156.0,1.0,11099.400000,71.150000,70.550000,3.451872,11.915419,62.600000,66.08500,66.500000,66.900000,68.400000,70.550000,73.425000,76.200000,78.200000,79.190000,79.900000
Length,156.0,1.0,29225.600000,187.343590,187.900000,13.431754,180.412023,149.400000,154.91500,163.675000,173.000000,177.575000,187.900000,196.125000,203.400000,208.500000,219.305000,224.500000


In [25]:
num_summary.to_csv('num_summary.csv')

In [28]:
#Creating Data Audit Report for Categorical
def cat_summary(x):
    return pd.Series([x.count(),x.isnull().sum(),x.value_counts()],index=['N','NMISS','ColumnNames'])
cate_summary=car_sales_cat.apply(lambda x:cat_summary(x))
cate_summary

,Latest_Launch,Model,Vehicle_type,Manufacturer
N,157,157,157,157
NMISS,0,0,0,0
ColumnNames,1/4/2012 2 4/24/2011 2 1/24/2011 ...,Neon 2 S40 1 Intrigue...,Passenger 116 Car 41 Name: Vehicl...,Ford 11 Dodge 11 Chevrolet ...


### Method 2

### Step 1

### Pandas Profiling:

In [29]:
pandas_profiling.ProfileReport(car_sales)

Number of variables,16
Number of observations,157
Total Missing (%),0.0%
Total size in memory,19.7 KiB
Average record size in memory,128.0 B
Numeric,10
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,2


### Step 2

#### Handling Outliers

In [31]:
car_sales.columns

Index([u'Manufacturer', u'Model', u'Sales_in_thousands',
       u'__year_resale_value', u'Vehicle_type', u'Price_in_thousands',
       u'Engine_size', u'Horsepower', u'Wheelbase', u'Width', u'Length',
       u'Curb_weight', u'Fuel_capacity', u'Fuel_efficiency', u'Latest_Launch',
       u'Power_perf_factor'],
      dtype='object')

In [34]:
#Handling Outliers:
#Method 1
car_sales['Sales_in_thousands']=car_sales['Sales_in_thousands'].clip_upper(car_sales['Sales_in_thousands'].quantile(.99))
car_sales['Sales_in_thousands']=car_sales['Sales_in_thousands'].clip_lower(car_sales['Sales_in_thousands'].quantile(.01))


In [35]:
car_summary=car_sales_num.apply(lambda x:var_summary(x)).T
car_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Curb_weight,155.0,2.0,523.594000,3.378026,3.342000,0.630502,0.397532,1.895000,2.24540,2.423500,2.606200,2.971000,3.342000,3.799500,4.118600,4.389100,5.396680,5.572000
Engine_size,156.0,1.0,477.500000,3.060897,3.000000,1.044653,1.091300,1.000000,1.55500,1.800000,1.950000,2.300000,3.000000,3.575000,4.600000,4.775000,5.700000,8.000000
Fuel_efficiency,154.0,3.0,3672.000000,23.844156,24.000000,4.282706,18.341567,15.000000,15.00000,16.650000,18.300000,21.000000,24.000000,26.000000,28.700000,31.000000,33.000000,45.000000
Price_in_thousands,155.0,2.0,4245.567000,27.390755,22.799000,14.351653,205.969949,9.235000,10.23144,12.469000,13.970800,18.017500,22.799000,31.947500,44.017000,55.835000,78.479800,85.500000
Power_perf_factor,155.0,2.0,11941.756636,77.043591,72.030917,25.142664,632.153558,23.276272,38.46192,46.203997,48.447741,60.407707,72.030917,89.414878,108.499252,125.091513,141.119461,188.144323
__year_resale_value,121.0,36.0,2186.830000,18.072975,14.180000,11.453384,131.180007,5.160000,6.17300,7.850000,9.125000,11.260000,14.180000,19.875000,32.075000,41.250000,60.220000,67.550000
Fuel_capacity,156.0,1.0,2800.500000,17.951923,17.200000,3.887921,15.115932,10.300000,11.90000,12.500000,13.200000,15.800000,17.200000,19.575000,23.200000,25.400000,30.900000,32.000000
Wheelbase,156.0,1.0,16768.000000,107.487179,107.000000,7.641303,58.389512,92.600000,92.87500,95.875000,98.650000,103.000000,107.000000,112.200000,115.500000,119.250000,134.375000,138.700000
Width,156.0,1.0,11099.400000,71.150000,70.550000,3.451872,11.915419,62.600000,66.08500,66.500000,66.900000,68.400000,70.550000,73.425000,76.200000,78.200000,79.190000,79.900000
Length,156.0,1.0,29225.600000,187.343590,187.900000,13.431754,180.412023,149.400000,154.91500,163.675000,173.000000,177.575000,187.900000,196.125000,203.400000,208.500000,219.305000,224.500000


In [36]:
#Handling Outliers:
#Method 2
def outlier_capping(x):
    x=x.clip_upper(x.quantile(.99))
    x=x.clip_lower(x.quantile(.01))
    return x
car_sales_num=car_sales_num.apply(lambda x:outlier_capping(x))

In [38]:
car_summary=car_sales_num.apply(lambda x:var_summary(x)).T
car_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Curb_weight,155.0,2.0,523.770160,3.379162,3.342000,0.621764,0.386591,2.24540,2.247884,2.423500,2.606200,2.971000,3.342000,3.799500,4.118600,4.389100,5.394693,5.396680
Engine_size,156.0,1.0,475.810000,3.050064,3.000000,0.982279,0.964871,1.55500,1.579750,1.800000,1.950000,2.300000,3.000000,3.575000,4.600000,4.775000,5.700000,5.700000
Fuel_efficiency,154.0,3.0,3660.000000,23.766234,24.000000,3.994755,15.958068,15.00000,15.000000,16.650000,18.300000,21.000000,24.000000,26.000000,28.700000,31.000000,33.000000,33.000000
Price_in_thousands,155.0,2.0,4235.955480,27.328745,22.799000,14.063970,197.795238,10.23144,10.476362,12.469000,13.970800,18.017500,22.799000,31.947500,44.017000,55.835000,76.584508,78.479800
Power_perf_factor,155.0,2.0,11911.685368,76.849583,72.030917,23.844968,568.582480,38.46192,39.285152,46.203997,48.447741,60.407707,72.030917,89.414878,108.499252,125.091513,141.109484,141.119461
__year_resale_value,121.0,36.0,2180.421000,18.020008,14.180000,11.181335,125.022260,6.17300,6.423400,7.850000,9.125000,11.260000,14.180000,19.875000,32.075000,41.250000,59.896000,60.220000
Fuel_capacity,156.0,1.0,2799.900000,17.948077,17.200000,3.819854,14.591287,11.90000,11.900000,12.500000,13.200000,15.800000,17.200000,19.575000,23.200000,25.400000,30.405000,30.900000
Wheelbase,156.0,1.0,16760.100000,107.436538,107.000000,7.424295,55.120156,92.87500,92.998750,95.875000,98.650000,103.000000,107.000000,112.200000,115.500000,119.250000,132.518750,134.375000
Width,156.0,1.0,11102.450000,71.169551,70.550000,3.390339,11.494400,66.08500,66.258250,66.500000,66.900000,68.400000,70.550000,73.425000,76.200000,78.200000,79.140500,79.190000
Length,156.0,1.0,29223.940000,187.332949,187.900000,13.120350,172.143594,154.91500,156.226750,163.675000,173.000000,177.575000,187.900000,196.125000,203.400000,208.500000,217.102250,219.305000


### Step 3

### Handling Missing Value

In [44]:
# Method 1
car_sales['Curb_weight']=car_sales['Curb_weight'].fillna(car_sales['Curb_weight'].mean())

In [47]:
car_summary=car_sales_num.apply(lambda x:var_summary(x)).T
car_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Curb_weight,157.0,0.0,530.350052,3.378026,3.368000,0.626447,0.392436,1.895000,2.245600,2.424000,2.612800,2.975000,3.368000,3.778000,4.117400,4.388400,5.396520,5.572000
Engine_size,156.0,1.0,477.500000,3.060897,3.000000,1.044653,1.091300,1.000000,1.555000,1.800000,1.950000,2.300000,3.000000,3.575000,4.600000,4.775000,5.700000,8.000000
Fuel_efficiency,154.0,3.0,3672.000000,23.844156,24.000000,4.282706,18.341567,15.000000,15.000000,16.650000,18.300000,21.000000,24.000000,26.000000,28.700000,31.000000,33.000000,45.000000
Price_in_thousands,155.0,2.0,4245.567000,27.390755,22.799000,14.351653,205.969949,9.235000,10.231440,12.469000,13.970800,18.017500,22.799000,31.947500,44.017000,55.835000,78.479800,85.500000
Power_perf_factor,155.0,2.0,11941.756636,77.043591,72.030917,25.142664,632.153558,23.276272,38.461920,46.203997,48.447741,60.407707,72.030917,89.414878,108.499252,125.091513,141.119461,188.144323
__year_resale_value,121.0,36.0,2186.830000,18.072975,14.180000,11.453384,131.180007,5.160000,6.173000,7.850000,9.125000,11.260000,14.180000,19.875000,32.075000,41.250000,60.220000,67.550000
Fuel_capacity,156.0,1.0,2800.500000,17.951923,17.200000,3.887921,15.115932,10.300000,11.900000,12.500000,13.200000,15.800000,17.200000,19.575000,23.200000,25.400000,30.900000,32.000000
Wheelbase,156.0,1.0,16768.000000,107.487179,107.000000,7.641303,58.389512,92.600000,92.875000,95.875000,98.650000,103.000000,107.000000,112.200000,115.500000,119.250000,134.375000,138.700000
Width,156.0,1.0,11099.400000,71.150000,70.550000,3.451872,11.915419,62.600000,66.085000,66.500000,66.900000,68.400000,70.550000,73.425000,76.200000,78.200000,79.190000,79.900000
Length,156.0,1.0,29225.600000,187.343590,187.900000,13.431754,180.412023,149.400000,154.915000,163.675000,173.000000,177.575000,187.900000,196.125000,203.400000,208.500000,219.305000,224.500000


In [49]:
#Method 2
def Missing_imputation(x):
    x=x.fillna(x.mean())
    return x
car_sales_num=car_sales_num.apply(lambda x:Missing_imputation(x))


In [50]:
car_summary=car_sales_num.apply(lambda x:var_summary(x)).T
car_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Curb_weight,157.0,0.0,530.350052,3.378026,3.368000,0.626447,0.392436,1.895000,2.245600,2.424000,2.612800,2.975000,3.368000,3.778000,4.11740,4.388400,5.396520,5.572000
Engine_size,157.0,0.0,480.560897,3.060897,3.000000,1.041299,1.084304,1.000000,1.556000,1.800000,1.960000,2.300000,3.000000,3.500000,4.60000,4.760000,5.700000,8.000000
Fuel_efficiency,157.0,0.0,3743.532468,23.844156,24.000000,4.241326,17.988844,15.000000,15.000000,16.800000,18.600000,21.000000,24.000000,26.000000,28.40000,31.000000,33.000000,45.000000
Price_in_thousands,157.0,0.0,4300.348510,27.390755,23.400000,14.259359,203.329309,9.235000,10.251160,12.491000,13.976200,18.145000,23.400000,31.930000,43.78800,55.225000,78.327200,85.500000
Power_perf_factor,157.0,0.0,12095.843819,77.043591,72.290355,24.980973,624.049025,23.276272,38.528203,46.257114,48.522793,60.727447,72.290355,89.401935,107.99432,125.065461,141.118657,188.144323
__year_resale_value,157.0,0.0,2837.457107,18.072975,17.710000,10.045282,100.907698,5.160000,6.736400,8.425000,9.710000,12.545000,17.710000,18.140000,27.02500,39.275000,59.491000,67.550000
Fuel_capacity,157.0,0.0,2818.451923,17.951923,17.200000,3.875440,15.019035,10.300000,11.900000,12.500000,13.200000,15.800000,17.200000,19.500000,23.20000,25.400000,30.880000,32.000000
Wheelbase,157.0,0.0,16875.487179,107.487179,107.000000,7.616772,58.015220,92.600000,92.880000,95.920000,98.700000,103.000000,107.000000,112.200000,115.46000,119.200000,134.300000,138.700000
Width,157.0,0.0,11170.550000,71.150000,70.600000,3.440790,11.839038,62.600000,66.092000,66.500000,66.900000,68.400000,70.600000,73.400000,76.18000,78.200000,79.188000,79.900000
Length,157.0,0.0,29412.943590,187.343590,187.800000,13.388635,179.255536,149.400000,154.968000,163.700000,173.120000,177.600000,187.800000,196.100000,203.34000,208.500000,219.216000,224.500000


### Step 4:

### Handling Categorical Variable:

In [55]:
#Create Dummies
col_dummies=pd.get_dummies(car_sales['Vehicle_type'],prefix='Vehicle_type')
col_dummies.head(3)

,Vehicle_type_Car,Vehicle_type_Passenger
0,0,1
1,0,1
2,0,1


In [56]:
col_dummies.drop(col_dummies.columns[0],axis=1,inplace=True)

In [57]:
col_dummies.head(3)

,Vehicle_type_Passenger
0,1
1,1
2,1


In [58]:
car_sales_cat.head(3)

,Latest_Launch,Model,Vehicle_type,Manufacturer
0,2/2/2012,Integra,Passenger,Acura
1,6/3/2011,TL,Passenger,Acura
2,1/4/2012,CL,Passenger,Acura


In [59]:
car_sales_cat=pd.concat([car_sales['Vehicle_type'],col_dummies],axis=1)

In [60]:
car_sales_cat.head(3)

,Vehicle_type,Vehicle_type_Passenger
0,Passenger,1
1,Passenger,1
2,Passenger,1


In [61]:
#Method 2
# An utility function to create dummy variables
def create_dummy(df,colname):
    col_dummies=pd.get_dummies(df[colname],prefix=colname)
    col_dummies.drop(col_dummies.columns[0],axis=1,inplace=True)
    df=pd.concat([df,col_dummies],axis=1)
    df.drop(colname,axis=1,inplace=True)
    return df

In [68]:
#For C-Feature in Categorical Features:
car_sales_cat=car_sales[['Manufacturer','Vehicle_type']]
for c_feature in ['Manufacturer','Vehicle_type']:
    car_sales_cat[c_feature]=car_sales_cat[c_feature].astype('category')
    car_sales_cat=create_dummy(car_sales_cat,c_feature)
    

C:\Users\Satya\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [71]:
car_sales_cat

,Manufacturer_Audi,Manufacturer_BMW,Manufacturer_Buick,Manufacturer_Cadillac,Manufacturer_Chevrolet,Manufacturer_Chrysler,Manufacturer_Dodge,Manufacturer_Ford,Manufacturer_Honda,Manufacturer_Hyundai,...,Manufacturer_Plymouth,Manufacturer_Pontiac,Manufacturer_Porsche,Manufacturer_Saab,Manufacturer_Saturn,Manufacturer_Subaru,Manufacturer_Toyota,Manufacturer_Volkswagen,Manufacturer_Volvo,Vehicle_type_Passenger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [72]:
#PUT THEM BACK TOGETHER Numerical and categorical
car_sales_new=pd.concat([car_sales_num,car_sales_cat],axis=1)


In [73]:
car_sales_new.head(3)

,Curb_weight,Engine_size,Fuel_efficiency,Price_in_thousands,Power_perf_factor,__year_resale_value,Fuel_capacity,Wheelbase,Width,Length,...,Manufacturer_Plymouth,Manufacturer_Pontiac,Manufacturer_Porsche,Manufacturer_Saab,Manufacturer_Saturn,Manufacturer_Subaru,Manufacturer_Toyota,Manufacturer_Volkswagen,Manufacturer_Volvo,Vehicle_type_Passenger
0,2.639,1.8,28.0,21.500000,58.280150,16.360,13.2,101.2,67.3,172.4,...,0,0,0,0,0,0,0,0,0,1
1,3.517,3.2,25.0,28.400000,91.370778,19.875,17.2,108.1,70.3,192.9,...,0,0,0,0,0,0,0,0,0,1
2,3.470,3.2,26.0,27.390755,77.043591,18.225,17.2,106.9,70.6,192.0,...,0,0,0,0,0,0,0,0,0,1


In [85]:
pandas_profiling.ProfileReport(car_sales_new)

Number of variables,42
Number of observations,157
Total Missing (%),0.0%
Total size in memory,19.4 KiB
Average record size in memory,126.0 B
Numeric,11
Categorical,0
Boolean,30
Date,0
Text (Unique),0
Rejected,1
